In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pandas as pd

C:\Users\LASYA\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
import keras.backend as K

In [3]:
def keras_model(image_x, image_y):
    num_of_classes = 12
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(image_x, image_y, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(64, (5, 5), activation='sigmoid'))
    model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = "emojinator.h5"
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint1]

    return model, callbacks_list

In [4]:
data = pd.read_csv("train_foo.csv")
dataset = np.array(data)
np.random.shuffle(dataset)
X = dataset
Y = dataset
X = X[:, 1:2501]
Y = Y[:, 0]

In [5]:
X_train = X[0:12000, :]
X_train = X_train / 255.
X_test = X[12000:13201, :]
X_test = X_test / 255.

In [6]:
# Reshape
Y = Y.reshape(Y.shape[0], 1)
Y_train = Y[0:12000, :]
Y_train = Y_train.T
Y_test = Y[12000:13201, :]
Y_test = Y_test.T

In [7]:
print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))
image_x = 50
image_y = 50

number of training examples = 12000
number of test examples = 1201
X_train shape: (12000, 2500)
Y_train shape: (1, 12000)
X_test shape: (1201, 2500)
Y_test shape: (1, 1201)


In [8]:
train_y = np_utils.to_categorical(Y_train)
test_y = np_utils.to_categorical(Y_test)
train_y = train_y.reshape(train_y.shape[1], train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])
X_train = X_train.reshape(X_train.shape[0], 50, 50, 1)
X_test = X_test.reshape(X_test.shape[0], 50, 50, 1)
print("X_train shape: " + str(X_train.shape))
print("X_test shape: " + str(X_test.shape))

X_train shape: (12000, 50, 50, 1)
X_test shape: (1201, 50, 50, 1)


In [9]:
model, callbacks_list = keras_model(image_x, image_y)
model.fit(X_train, train_y, validation_data=(X_test, test_y), epochs=10, batch_size=64,callbacks=callbacks_list)
scores = model.evaluate(X_test, test_y, verbose=0)
print("CNN Error: %.2f%%" % (100 - scores[1] * 100))

model.save('emojinator.h5')

W0816 15:33:11.008478  2704 deprecation_wrapper.py:119] From C:\Users\LASYA\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0816 15:33:12.491786  2704 deprecation_wrapper.py:119] From C:\Users\LASYA\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0816 15:33:12.781532  2704 deprecation_wrapper.py:119] From C:\Users\LASYA\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0816 15:33:13.033196  2704 deprecation_wrapper.py:119] From C:\Users\LASYA\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0816 15:33:13.351627  2704 deprecation_wrapper.py:119] From C:\Users\LASYA\Ana

Train on 12000 samples, validate on 1201 samples
Epoch 1/10
12000/12000 [==============================] - 70s 6ms/step - loss: 0.5237 - acc: 0.8426 - val_loss: 0.0061 - val_acc: 0.9975

Epoch 00001: val_acc improved from -inf to 0.99750, saving model to emojinator.h5
Epoch 2/10
12000/12000 [==============================] - 55s 5ms/step - loss: 0.0089 - acc: 0.9979 - val_loss: 5.5873e-04 - val_acc: 1.0000

Epoch 00002: val_acc improved from 0.99750 to 1.00000, saving model to emojinator.h5
Epoch 3/10
12000/12000 [==============================] - 57s 5ms/step - loss: 0.0032 - acc: 0.9994 - val_loss: 1.2900e-04 - val_acc: 1.0000

Epoch 00003: val_acc did not improve from 1.00000
Epoch 4/10
12000/12000 [==============================] - 60s 5ms/step - loss: 0.0026 - acc: 0.9993 - val_loss: 8.3555e-04 - val_acc: 1.0000

Epoch 00004: val_acc did not improve from 1.00000
Epoch 5/10
12000/12000 [==============================] - 50s 4ms/step - loss: 0.0016 - acc: 0.9997 - val_loss: 2.9049e-